<a href="https://colab.research.google.com/github/nibin101/intelligent_quiz/blob/main/intelligent_quiz_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STEP 0: Install dependencies

# STEP 1: Imports and secure API key input

In [31]:
import os
import getpass


# STEP 2: Initialize Gemini 1.5 Flash

In [32]:
os.environ['GOOGLE_API_KEY']=getpass.getpass("Enter the Gemini api key")

Enter the Gemini api key··········



# Secure Gemini API Key input

In [27]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model="models/gemini-1.5-flash-latest" ,temperature=0.2)

# STEP 3: Node to ask user for symptom

In [28]:
def get_subject_input(state: dict) -> dict:
    """Gets the subject input from the user."""
    subject = input("What subject do you want to learn about (physics, chemistry, or maths)? ")
    state["subject"] = subject.lower()
    return state

# STEP 4: Node to classify the symptom

In [29]:
import json

def generate_mcqs(state: dict) -> dict:
    """Generates 5 MCQ questions on the given subject and stores them in the state."""
    subject = state["subject"]
    prompt = (
        f"Generate 5 multiple-choice questions about {subject}. "
        "Each question should have 4 options (A, B, C, D) and indicate the correct answer. "
        "Respond in JSON format as a list of objects, where each object has keys 'question', 'options', and 'answer'."
        "Do not include any ''' formatting"
    )
    response = llm.invoke([HumanMessage(content=prompt)])
    print(response.content)
    try:
        # Attempt to parse the content directly first
        questions_json = json.loads(response.content)
        state["questions"] = [(q["question"], q["options"], q["answer"]) for q in questions_json]
    except json.JSONDecodeError as e:
        print("Error decoding JSON response from the language model.")
        print(e)
        # If direct parsing fails, try stripping potential markdown formatting
        try:
            cleaned_response = response.content.strip().replace("```json\n", "").replace("```", "")
            questions_json = json.loads(cleaned_response)
            state["questions"] = [(q["question"], q["options"], q["answer"]) for q in questions_json]
        except json.JSONDecodeError as e_cleaned:
            print("Error decoding JSON response after cleaning.")
            print(e_cleaned)
            state["questions"] = []
    return state

# STEP 5: Router logic to route to the correct node


In [34]:
import json
import re
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

def get_subject_input(state: dict) -> dict:
    """Gets the subject input from the user."""
    subject = input("What subject do you want to learn about and be quizzed on? ")
    state["subject"] = subject.lower()
    return state

def generate_mcqs(state: dict) -> dict:
    """Generates 5 MCQ questions on the given subject and stores them in the state."""
    subject = state["subject"]
    prompt = (
        f"Generate 5 multiple-choice questions about {subject} at a mid average difficulty level. "
        "Each question should have 4 options (A, B, C, D) and indicate the correct answer. "
        "Respond in JSON format as a list of objects, where each object has keys 'question', 'options', and 'answer'."
        "Do not include any ''' formatting"
    )
    # Ensure llm is defined or imported if needed here
    # response = llm.invoke([HumanMessage(content=prompt)]) # Uncomment and ensure llm is available

    # Mock response for demonstration if llm is not available
    # response_content = """
    # [
    #     {
    #         "question": "What is the chemical symbol for water?",
    #         "options": {"A": "O2", "B": "H2O", "C": "CO2", "D": "NaCl"},
    #         "answer": "B"
    #     },
    #     {
    #         "question": "What is the capital of France?",
    #         "options": {"A": "Berlin", "B": "Madrid", "C": "Paris", "D": "Rome"},
    #         "answer": "C"
    #     },
    #     {
    #         "question": "What is the largest planet in our solar system?",
    #         "options": {"A": "Earth", "B": "Mars", "C": "Jupiter", "D": "Venus"},
    #         "answer": "C"
    #     },
    #     {
    #         "question": "What is the square root of 64?",
    #         "options": {"A": "6", "B": "7", "C": "8", "D": "9"},
    #         "answer": "C"
    #     },
    #     {
    #         "question": "Who wrote 'Romeo and Juliet'?",
    #         "options": {"A": "Charles Dickens", "B": "William Shakespeare", "C": "Jane Austen", "D": "Leo Tolstoy"},
    #         "answer": "B"
    #     }
    # ]
    # """
    # Replace the mock response with the actual LLM call when available
    try:
        # llm=ChatGoogleGenerativeAI(model="models/gemini-1.5-flash-latest" ,temperature=0.2) # Removed redundant initialization
        response = llm.invoke([HumanMessage(content=prompt)])
        questions_json = json.loads(response.content)
        state["questions"] = [(q["question"], q["options"], q["answer"]) for q in questions_json]
    except json.JSONDecodeError as e:
        print("Error decoding JSON response from the language model.")
        print(e)
        try:
            cleaned_response = response.content.strip().replace("```json\n", "").replace("```", "")
            questions_json = json.loads(cleaned_response)
            state["questions"] = [(q["question"], q["options"], q["answer"]) for q in questions_json]
        except json.JSONDecodeError as e_cleaned:
            print("Error decoding JSON response after cleaning.")
            print(e_cleaned)
            state["questions"] = []

    # Using mock response for now
    # try:
    #     questions_json = json.loads(response_content)
    #     state["questions"] = [(q["question"], q["options"], q["answer"]) for q in questions_json]
    # except json.JSONDecodeError as e:
    #     print("Error decoding mock JSON response.")
    #     print(e)
    #     state["questions"] = []

    return state


def main():
    state = {}
    state = get_subject_input(state)

    while True:
        state = generate_mcqs(state)

        correct_answers = 0
        total_questions = len(state.get("questions", []))
        answered_correctly = []
        answered_incorrectly = []

        # Display MCQs and get user answers
        if state.get("questions"):
            print("\nHere are the questions:")
            for i, (question, options, answer) in enumerate(state["questions"]):
                # Remove <sup> tags from the question
                cleaned_question = re.sub(r'<sup[^>]*>.*?</sup>', '', question)
                print(f"\nQuestion {i+1}: {cleaned_question}")
                # Iterate through the options dictionary to display letter and text
                for letter, option_text in options.items():
                    # Remove <sup> tags from the options
                    cleaned_option_text = re.sub(r'<sup[^>]*>.*?</sup>', '', option_text)
                    print(f"{letter}. {cleaned_option_text}")

                user_answer = input("Your answer (enter the option letter, e.g., A): ").upper()

                # Check if the answer is correct
                # Simplified check based on the dictionary answer
                correct_option_letter = answer


                if user_answer == correct_option_letter:
                    print("Correct!")
                    correct_answers += 1
                    answered_correctly.append(question)
                else:
                    print(f"Incorrect. The correct answer is {correct_option_letter}.")
                    answered_incorrectly.append(question)
        else:
            print("No questions were generated.")

        # Print the score
        if total_questions > 0:
            print(f"\nYour score: {correct_answers}/{total_questions}")

            # Use LLM to summarize topics
            if answered_incorrectly:
                print("\nHere's a summary of the topics you answered incorrectly:")
                for incorrect_question in answered_incorrectly:
                    incorrect_topic_prompt = f"Explain the topic of the following question in a short paragraph:\n{incorrect_question}"
                    llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash-latest" ,temperature=0.2)
                    incorrect_topic_response = llm.invoke([HumanMessage(content=incorrect_topic_prompt)])
                    cleaned_incorrect_topic_explanation = incorrect_topic_response.content.replace('**', '')
                    print(f"\n{cleaned_incorrect_topic_explanation}")

            # Ask for the next topic
            next_subject = input("\nWhich topic would you like to be quizzed on next? (or type 'quit' to exit) ")
            if next_subject.lower() == 'quit':
                break
            state["subject"] = next_subject.lower()
            print(f"Okay, I will generate questions about {next_subject} next.")


if __name__ == "__main__":
    main()

What subject do you want to learn about and be quizzed on? maths
Error decoding JSON response from the language model.
Expecting value: line 1 column 1 (char 0)

Here are the questions:

Question 1: What is the value of x in the equation 3x + 7 = 16?
A. 3
B. 6
C. 9
D. 11
Your answer (enter the option letter, e.g., A): b
Incorrect. The correct answer is A.

Question 2: A rectangle has a length of 12cm and a width of 5cm. What is its area?
A. 17 cm²
B. 26 cm²
C. 60 cm²
D. 70 cm²


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-850116287.py", line 149, in <cell line: 0>
    main()
  File "/tmp/ipython-input-850116287.py", line 109, in main
    user_answer = input("Your answer (enter the option letter, e.g., A): ").upper()
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 1177, in raw_input
    return self._input_request(
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 1219, in _input_request
    raise KeyboardInterrupt("Interrupted by user") from None
KeyboardInterrupt: Interrupted by user

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/

TypeError: object of type 'NoneType' has no len()

# STEP 6: Category-specific response nodes


In [ ]:
def general_node(stae:dict)->dict
  state["answer"]=f"{state['state']} sems to be general .you are being directed to cunsulting a general doctor"



# STEP 7: Build LangGraph


# STEP 8: Compile and invoke the graph
